# How to handle "double-texting" or concurrent runs in your graph

You might want to start a new run on a thread while the previous run still haven't finished. We call this "double-texting" or multi-tasking.

There are several strategies for handling this:
 
- `reject`: Reject the new run.
- `interrupt`: Interrupt the current run, keeping steps completed until now, and start a new one.
- `rollback`: Cancel and delete the existing run, rolling back the thread to the state before it had started, then start the new run.
- `enqueue`: Queue up the new run to start after the current run finishes.

### Reject

In [1]:
from langgraph_sdk import get_client
from langchain_core.messages import convert_to_messages
import httpx

In [2]:
client = get_client()

In [3]:
assistant = await client.assistants.create("agent")

In [4]:
thread = await client.threads.create()

In [5]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [6]:
try:
    await client.runs.create(
        thread["thread_id"],
        assistant["assistant_id"],
        input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/f9e7088b-8028-4e5c-88d2-9cc9a2870e50/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


We can verify that the original thread finished executing:

In [7]:
# wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

In [8]:
state = await client.threads.get_state(thread["thread_id"])

In [9]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01CyewEifV2Kmi7EFKHbMDr1', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01CyewEifV2Kmi7EFKHbMDr1)
 Call ID: toolu_01CyewEifV2Kmi7EFKHbMDr1
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629", "content": "Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead."}]
================================== Ai Message ==================================

According to the search results from Tavily, the current weather in San Franci

### Interrupt

In [10]:
import asyncio

In [11]:
thread = await client.threads.create()

In [12]:
# the first run will be interrupted
interrupted_run = await client.runs.create(
    thread["thread_id"], assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]},
)
await asyncio.sleep(2)
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="interrupt",
)

In [13]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has partial data from the first run + data from the second run

In [14]:
state = await client.threads.get_state(thread["thread_id"])

In [15]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01MjNtVJwEcpujRGrf3x6Pih', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01MjNtVJwEcpujRGrf3x6Pih)
 Call ID: toolu_01MjNtVJwEcpujRGrf3x6Pih
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/KCASANFR2002/date/2024-6-18", "content": "High 64F. Winds W at 10 to 20 mph. A few clouds from time to time. Low 49F. Winds W at 10 to 20 mph. Temp. San Francisco Weather Forecasts. Weather Underground provides local & long-range weather ..."}]
================================ Human Message ==============================

Verify that the original, interrupted run was interrupted

In [16]:
(await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"]

'interrupted'

### Rollback

In [17]:
thread = await client.threads.create()

In [18]:
# the first run will be interrupted
rolled_back_run = await client.runs.create(
    thread["thread_id"], assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]},
)
await asyncio.sleep(2)
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="rollback",
)

In [19]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has data only from the second run

In [20]:
state = await client.threads.get_state(thread["thread_id"])

In [21]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in nyc?
================================== Ai Message ==================================

[{'id': 'toolu_01JzPqefao1gxwajHQ3Yh3JD', 'input': {'query': 'weather in nyc'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01JzPqefao1gxwajHQ3Yh3JD)
 Call ID: toolu_01JzPqefao1gxwajHQ3Yh3JD
  Args:
    query: weather in nyc
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.71, 'lon': -74.01, 'tz_id': 'America/New_York', 'localtime_epoch': 1718734479, 'localtime': '2024-06-18 14:14'}, 'current': {'last_updated_epoch': 1718733600, 'last_updated': '2024-06-18 14:00', 'temp_c': 29.4, 'temp_f': 84.9, 'is_day': 1, 'condition

Verify that the original, rolled back run was deleted

In [22]:
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as e:
    print("Original run was correctly deleted")

Original run was correctly deleted


### Enqueue

In [23]:
thread = await client.threads.create()

In [24]:
# this run will be interrupted
first_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [25]:
second_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="enqueue",
)

Verify that the thread has data from both runs

In [26]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

In [27]:
state = await client.threads.get_state(thread["thread_id"])

In [28]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01Dez1sJre4oA2Y7NsKJV6VT', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01Dez1sJre4oA2Y7NsKJV6VT)
 Call ID: toolu_01Dez1sJre4oA2Y7NsKJV6VT
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629", "content": "Get the current and future weather conditions for San Francisco, CA, including temperature, precipitation, wind, air quality and more. See the hourly and 10-day outlook, radar maps, alerts and allergy information."}]
================================== Ai Message =======================